In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
print("Setup Complete")

# **Missing Values and Cleaning Data**

> The first thing after after we download the file is to check for any missing values, which will be reprsented with "NaN" or "None".

In [1]:
df = pd.read_csv('../input/dataatp/data_atp.csv')
df.head()

To print a concise summary of a Tennis DataFrame and also to check the data type of our date column

In [1]:
df.info()

Convert our date columns to datetime.

In [1]:
df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%y")

Eleiminate the last 8 columns, since they contain no values.

In [1]:
df = df.drop(columns=df.iloc[:,28:36])

Fill "NaN" values with "0".

In [1]:
df.fillna(value=0, inplace=True)

Creating additional columns for number of Games per Set (GpS), and calculating Total number of games (Total) and a number of Sets played per match (Sets).

In [1]:
for i in range(1,6):
  df['GpS'+str(i)] = df.apply(lambda x: x['W'+str(i)] + x['L'+str(i)], axis = 1)

df['Total'] = 0

for i in range(1,6):
  df['Total'] += df['GpS'+str(i)]

df['Sets'] = df.apply(lambda x: x['Wsets'] + x['Lsets'] if x['Comment']=='Completed' else 0, axis=1)

In [1]:
df.head()

# **Data Visualization**

Create a sub-Dataframe containing the required data for the specific match. Find winning and losing matches for given players.

In [1]:
player1, player2 = 'Nadal R.','Djokovic N.'

print(player1, player2)

In [1]:
df1 = df.loc[(df['Series']!='Grand Slam') & (df['Sets'] > 1) & (df['Total'] > 0) & (df['Surface']!='Carpet') & (df['Comment'] == 'Completed')]

In [1]:
print(df1.groupby(['Winner','Sets'])[['Total']].describe().loc[[player1, player2]])
print(df1.groupby(['Loser','Sets'])[['Total']].describe().loc[[player1, player2]])

Combining the winning and losing data of two players and generating a kde smoothed distribution.

In [1]:
df2 =df1[['Total','Sets']].loc[(df1['Winner']==player1) | (df1['Loser']==player1) | (df1['Winner']==player2) | (df1['Loser']==player2)]
print(df2.groupby(['Sets'])[['Total']].describe())

In [1]:
plt.figure(figsize=(16,6))
sns.kdeplot(data=df2, x ='Total', shade=False, bw_adjust=3)
sns.kdeplot(data=df2, x ='Total', hue = 'Sets', shade=True, bw_adjust=2)
plt.show()

In [1]:
plt.figure(figsize=(16,6))
sns.kdeplot(data=df1, x ='Total', hue = 'Series', shade=True)
plt.show()